In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 29.4 MB/s eta 0:00:00


# Generate data for retriever to run

In [ ]:
from datasets import load_dataset

ds = load_dataset("neulab/odex", "en", split="test")

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
ids = ds['task_id']
intents = ds['intent']

In [ ]:
file_path = "output.txt"

# Write the list of strings to the file
with open(file_path, 'w') as file:
    for string in intents:
        file.write(f"{string}\n")

In [ ]:
file_path = "output.id"

# Write the list of strings to the file
with open(file_path, 'w') as file:
    for string in ids:
        file.write(f"{string}\n")

Now run the command for retriever inference which will generate retrieval_results.json

# Generate data for generator

In [ ]:
import json
file_path = 'retrieval_results.json'
with open(file_path, 'r') as file:
    data_dictionary = json.load(file)

In [ ]:
file_path = 'python_manual_firstpara.tok.txt'
with open(file_path, 'r') as file:
    content_list = file.readlines()
content_list = [line.strip() for line in content_list]
#target file

In [ ]:
file_path = 'python_manual_firstpara.tok.id'
with open(file_path, 'r') as file:
    IDs = file.readlines()
id_list = [line.strip() for line in IDs]
result_dict = dict(zip(id_list, content_list))

In [ ]:
# Create a new dictionary with modified keys
data_dictionary_new = {key.split('-')[0]: value for key, value in data_dictionary.items()}

# Update the original dictionary with the modified keys
data_dictionary.update(data_dictionary_new)

# Print the modified dictionary
print(data_dictionary_new)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
final_list= []
for data in ds:

    temp_dict = dict()

    temp_dict['id'] = str(data['task_id'])
    temp_dict['question'] = data['intent']
    temp_dict['target'] = data['canonical_solution']
    temp_dict['answers'] = [data['canonical_solution']]


    ele = data_dictionary[str(data['task_id'])]

    combined_list = list(zip(ele['retrieved'], ele['score']))
    sorted_combined_list = sorted(combined_list, key=lambda x: x[1], reverse=True)
    top_10_elements = sorted_combined_list[:10]
    result_strings,_ = zip(*top_10_elements)
    net_list = []
    for string in result_strings:
        tempd = dict()
        tempd['title'] = string.replace('.', ' ')
        tempd['text'] = result_dict[string]
        tempd['man_id'] = string
        net_list.append(tempd)


    temp_dict['ctxs'] = net_list
    final_list.append(temp_dict)

In [ ]:
import json


# Write the list into a JSON file
with open('test_output.json', 'w') as file:
    # Use json.dump to write the list to the file
    json.dump(final_list, file, indent=2)

Now run the command for generator inference with eval_data as test_output. The results will be stored in test_results_test_same.json